In [24]:
import random
from copy import deepcopy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import interpolate
from scipy.stats import spearmanr
from sklearn.preprocessing import robust_scale

In [25]:
data = pd.read_csv('data/1.data.csv', index_col=0)

In [26]:
data.head(25)

,DG 96910 - 3 MUSKETEERS FS 6PK 2.93OZ 24CA,DG 96910 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT,DG 96910 - 3 MUSKETEERS SINGLES 1.92OZ 36CT 10/CA,DG 96910 - COMBOS CHEDDAR PRETZEL SNGL 18CT 12/CA,DG 96910 - COMBOS PEPP PIZZA CRKR SNGL 18CT 12/CA,DG 96910 - COMBOS PIZZA PRETZEL SINGLES 18CT 12/CA,DG 96910 - DOVE MILK CHOC SINGLES 1.44OZ 18CT 12/CA,DG 96910 - M&MS MC MINIS MEGATUBE 1.77OZ 24CT 144CS,DG 96910 - M&MS MILK CHOC KINGSIZE 3.14OZ 24CT 6/CA,DG 96910 - M&MS PEANUT KING SIZE 3.27OZ 24CT 6/CA,...,DG DC 96970 - SKITTLES ORIGINAL SINGLES 36CT 10/CA,DG DC 96970 - SKITTLES SOURS SINGLES 1.8OZ 24CT 12/CA,DG DC 96970 - SKITTLES WILD BERRY SINGLES 36CT 10/CA,DG DC 96970 - STARBURST 14OZ LDBG 12/CA,DG DC 96970 - STARBURST FAVEREDS 14OZ LD BG 12/CA,DG DC 96970 - STARBURST ORIGINAL SINGLES 36CT 10/CA,DG DC 96970 - STB ORG JELBN BAG 14OZ 12/CS,DG DC 96970 - TWIX CARAMEL KS 3.02OZ 24CT 6/CA,DG DC 96970 - TWIX CARAMEL SNGL 1.79OZ 36CT 10/CA,DG DC 96970 - TWIX FS 6PK 3.28OZ 24CA
2016-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
tempDictionary = {}
for column in list(data):
    tempArray = tuple([str(value) for value in data[column].values])
    
    if tempArray not in tempDictionary.keys():
        tempDictionary[tempArray] = [column]
    
    else:
        tempDictionary[tempArray].append(column)

for key, value in tempDictionary.items():
    
    if len(value) > 1:
        data.drop(value, axis = 1, inplace = True)
        
        value = [subValue.split('|') for subValue in value]
        print(len(value), value)
        
        value = [item for subValue in value for item in subValue]
        data['|'.join(sorted(list(set(value))))] = [float(value) for value in key]
        
        print(data.values.shape)

3 [['DG DC 6150 - 3 MUSKETEERS FS 6PK 2.93OZ 24CA'], ['DG DC 6150 - MILKY WAY FS 6PK 3.36OZ 24CA'], ['DG DC 6150 - MILKYWAY SMP CARAMEL FS 6PK 4.42OZ 24CA']]
(1079, 456)
2 [['DG DC 6150 - 3 MUSKETEERS SINGLES 1.92OZ 36CT 10/CA'], ['DG DC 6150 - COMBOS PIZZA PRETZEL SINGLES 18CT 12/CA']]
(1079, 455)
2 [['DG DC 6150 - COMBOS CHEDDAR PRETZEL SNGL 18CT 12/CA'], ['DG DC 6150 - MILKY WAY SIMPLY CRML SNGL 1.91OZ 12/24C']]
(1079, 454)
2 [['DG DC 6150 - M&MS MILK CHOC KINGSIZE 3.14OZ 24CT 6/CA'], ['DG DC 6150 - MILKY WAY SINGLES 1.84OZ 36CT 10/CA']]
(1079, 453)
2 [['DG DC 6150 - MILKY WAY MIDNIGHT SINGLE 1.76OZ 12/24CT'], ['DG DC 6150 - TWIX CARAMEL SNGL 1.79OZ 36CT 10/CA']]
(1079, 452)
2 [['DG DC 6150 - MMS MC $1.00 PEG PK 2.83OZ 12/CA'], ['DG DC 6150 - MMS PEANUT $1.00 PEG PK 2.83OZ 12/CA']]
(1079, 451)


In [28]:
trajectoriesSet, trajectoriesSmoothOriginal, trajectoriesRaw = {}, {}, {}

for column in list(data):
    
    if column == 'index':
        continue
    
    tempDF = data[[column]]
    
    if len(list(set([value for value in tempDF[column].values if np.isfinite(value)]))) == 1:
        continue
    
    tempDF.index = pd.to_datetime(tempDF.index, yearfirst = True)
    
    tempDF = tempDF.resample('W').agg(np.nansum)
    tempDF.replace(0, np.nan, inplace = True)
    
    firstIndex = tempDF.first_valid_index()
    lastIndex = tempDF.last_valid_index()
    tempDFCut = tempDF.loc[firstIndex: lastIndex]
    
    tempDFCut = tempDFCut.ewm(span=len(tempDFCut.index) * 3).mean()
    tempDF = tempDF.ewm(span=len(tempDFCut.index) * 3).mean()
    
    trajectoriesSet[column] = [list(robust_scale(tempDFCut[column].values))]
    trajectoriesSmoothOriginal[column] = list(tempDF[column].values)# robust_scale(list(tempDF[column].values))
    trajectoriesRaw[column] = list(data[column].fillna(0))
    
    '''if len(trajectoriesSet.keys()) > 100:
        break'''

In [29]:
maxLength = max([len(value[0]) for _, value in trajectoriesSet.items()])

In [30]:
trajectoriesSetProcessed = {}
for key, value in trajectoriesSet.items():
    value = value[0]
    
    if len(value) == maxLength:
        trajectoriesSetProcessed[(key, )] = [value]
        continue
    
    oldScale = np.arange(0, maxLength, maxLength / len(value))
    
    oldScale = oldScale[: min([len(oldScale), len(value)])]
    value = value[: min([len(oldScale), len(value)])]
    
    try:
        interpolationFunction = interpolate.interp1d(oldScale, value)
    
    except:
        print(value)
        
        import sys
        sys.exit(0)
    
    cutOff = 0
    while True:
        newScale = np.linspace(0, maxLength - cutOff, maxLength)
    
        try:
            value = interpolationFunction(newScale)
            break
            
        except:
            cutOff += 1
    
    trajectoriesSetProcessed[(key, )] = [value]

In [31]:
THRESHOLD = 0.50

trajectories = deepcopy(trajectoriesSetProcessed)
distanceMatrixDictionary = {}

iteration = 1
while True:
    distanceMatrix = np.empty((len(trajectories), len(trajectories),))
    distanceMatrix[:] = np.nan
    
    for index1, (filter1, trajectory1) in enumerate(trajectories.items()):
        tempArray = []
        
        for index2, (filter2, trajectory2) in enumerate(trajectories.items()):
            
            if index1 > index2:
                continue
            
            elif index1 == index2:
                continue
            
            else:
                unionFilter = filter1 + filter2
                sorted(unionFilter)
                
                if unionFilter in distanceMatrixDictionary.keys():
                    distanceMatrix[index1][index2] = distanceMatrixDictionary.get(unionFilter)
                    
                    continue
                
                metric = []
                for subItem1 in trajectory1:
                    length1 = len(subItem1)
                    
                    for subItem2 in trajectory2:                        
                        metric.append(-spearmanr(subItem1, subItem2)[0] + 1)
                
                metric = max(metric)
                
                distanceMatrix[index1][index2] = metric
                distanceMatrixDictionary[unionFilter] = metric
    
    minValue = np.min(list(distanceMatrixDictionary.values()))
    
    print(minValue)
    
    if minValue > THRESHOLD:
        print(minValue, THRESHOLD)
        break
    
    minIndices = np.where(distanceMatrix == minValue)
    minIndices = list(zip(minIndices[0], minIndices[1]))
    
    minIndex = minIndices[0]
    
    filter1 = list(trajectories.keys())[minIndex[0]]
    filter2 = list(trajectories.keys())[minIndex[1]]
    
    trajectory1 = trajectories.get(filter1)
    trajectory2 = trajectories.get(filter2)
    
    unionFilter = filter1 + filter2
    sorted(unionFilter)
    
    trajectoryGroup = trajectory1 + trajectory2
    
    trajectories = {key: value for key, value in trajectories.items()
                    if all(value not in unionFilter for value in key)}
    
    distanceMatrixDictionary = {key: value for key, value in distanceMatrixDictionary.items()
                                if all(value not in unionFilter for value in key)}
    
    trajectories[unionFilter] = trajectoryGroup
    
    print(iteration, 'finished!')
    iteration += 1
    
    if len(list(trajectories.keys())) == 1:
        break

0.0019680874789799185
1 finished!
0.0025834550365591014
2 finished!
0.003741753687763083
3 finished!
0.004857306908615966
4 finished!
0.004895258433463612
5 finished!
0.005783982820396605
6 finished!
0.006566671579657957
7 finished!
0.006603297563709032
8 finished!
0.00713999881703109
9 finished!
0.007586862808455552
10 finished!
0.007656355754826261
11 finished!
0.007852467259530593
12 finished!
0.008100193731477079
13 finished!
0.009372207894633777
14 finished!
0.009481437055516406
15 finished!
0.010049178310143847
16 finished!
0.010614605485291362
17 finished!
0.010805820309299219
18 finished!
0.010964221460043944
19 finished!
0.011408928806923457
20 finished!
0.013303911341123964
21 finished!
0.013310259316498652
22 finished!
0.013648294930720284
23 finished!
0.013725478762033672
24 finished!
0.014291681095948028
25 finished!
0.01468345357251255
26 finished!
0.015051484313874375
27 finished!
0.015061323356917233
28 finished!
0.01614311836266913
29 finished!
0.016248414647364817
30 

0.19006254480330553
245 finished!
0.19114821207650312
246 finished!
0.19200229061449303
247 finished!
0.19401640780814688
248 finished!
0.19446976568260144
249 finished!
0.1958673546086276
250 finished!
0.19619281880512185
251 finished!
0.19736618866447808
252 finished!
0.19777617359334543
253 finished!
0.19875207375413473
254 finished!
0.1987784745273109
255 finished!
0.19974459710751447
256 finished!
0.2012193038162745
257 finished!
0.20256989926884306
258 finished!
0.20471051778849347
259 finished!
0.20514192331934045
260 finished!
0.21008352373295303
261 finished!
0.21110951486652574
262 finished!
0.2125182512275854
263 finished!
0.2137083222060615
264 finished!
0.21455543199729254
265 finished!
0.21805210337630954
266 finished!
0.21839487176897
267 finished!
0.219014922852803
268 finished!
0.2197080854278941
269 finished!
0.2199984366219544
270 finished!
0.22086580581622395
271 finished!
0.22341228635751686
272 finished!
0.22401231240555797
273 finished!
0.22607124151298508
274 fi

In [32]:
for key, value in trajectories.items():
    print(key, len(value))

('DG 96910 - DOVE MILK CHOC SINGLES 1.44OZ 18CT 12/CA',) 1
('DG 96920 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT',) 1
('DG DC 6300 - DOVE MILK CHOC SINGLES 1.44OZ 18CT 12/CA',) 1
('DG DC 6930 - MILKY WAY SIMPLY CRML SNGL 1.91OZ 12/24C', 'DG DC 6140 - M&MS MILK CHOC KINGSIZE 3.14OZ 24CT 6/CA', 'DG DC 6930 - SKITTLES SOURS SINGLES 1.8OZ 24CT 12/CA') 3
('DG DC 6500 - STARBURST ORIGINAL SINGLES 36CT 10/CA', 'DG DC 96970 - STB ORG JELBN BAG 14OZ 12/CS') 2
('DG DC 6800 - MILKY WAY SIMPLY CRML SNGL 1.91OZ 12/24C', 'DG DC 96970 - MILKYWAY SMP CARAMEL FS 6PK 4.42OZ 24CA') 2
('DG DC 6900 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT', 'DG DC 6930 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT', 'DG DC 6120 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT', 'DG DC 6300 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT') 4
('DG DC 6900 - TWIX FS 6PK 3.28OZ 24CA', 'DG DC 6000 - TWIX FS 6PK 3.28OZ 24CA', 'DG DC 96970 - 3 MUSKETEERS FS 6PK 2.93OZ 24CA') 3
('DG 96910 - MILKYWAY SMP CARAMEL FS 6PK 4.42OZ 24CA', 'DG DC 613

In [33]:
clusterNames = sorted(list(set(trajectories.keys())))

nameColumn, idColumn = [], []
for clusterIndex, clusterName in enumerate(clusterNames):
    
    if len(clusterName) == 1:
        continue
    
    nameColumn.append('|'.join(clusterName))
    idColumn.append(clusterIndex + 1)

resultDF = pd.DataFrame()
resultDF['name'] = nameColumn
resultDF['id'] = idColumn

resultDF.to_csv('2.1.clusteringWeekly[correlation+interpolation].csv')

In [34]:
for clusterIndex, clusterName in enumerate(clusterNames):
    
    if len(clusterName) == 1:
        continue
    
    figure = make_subplots(rows=3, cols=1)
    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(clusterName))]
    
    value = []
    for subKey in clusterName:
        value.append(trajectoriesSetProcessed.get((subKey, ))[0])
    
    for index, subValue in enumerate(value):
        figure.add_trace(go.Scatter(x=list(range(0, len(subValue))), y=subValue,
                                    mode='lines', marker_color=colors[index], line = dict(width=2.5), line_shape='spline'), row=1, col=1)
    
    #
    
    value = []
    for subKey in clusterName:
        value.append(trajectoriesSmoothOriginal.get(subKey))
    
    for index, subValue in enumerate(value):
        figure.add_trace(go.Scatter(x=list(range(0, len(subValue))), y=subValue,
                                    mode='lines', marker_color=colors[index], line = dict(width=2.5), line_shape='spline'), row=2, col=1)
    
    value = []
    for subKey in clusterName:
        value.append(trajectoriesRaw.get(subKey))
        # value.append(trajectoriesSetProcessed.get((subKey, ))[0])
    
    for index, subValue in enumerate(value):
        figure.add_trace(go.Scatter(x=list(range(0, len(subValue))), y=subValue,
                                    mode='lines', marker_color=colors[index], line = dict(width=2.5), line_shape='spline'), row=3, col=1)
    
    #    
    
    figure.update_layout(title = 'Cluster ' + str(clusterIndex + 1), showlegend=False, height=1200, width=1200)
    # figure.show()
    
    figure.write_image('images/cluster_' + str(clusterIndex + 1) + '.png')
    
    # break